_Note: This notebook requires DBR 16.2 or higher if executed against a classic compute cluster.  Databricks Serverless SQL Warehouse is preferred. Unity Catalog must be enabled with catalog and schema that the "run as" user has the ability to create and execute functions._

In [0]:
DECLARE OR REPLACE VARIABLE secret_scope STRING DEFAULT REPLACE(SPLIT(current_user(), "@")[0], ".", "-");
DECLARE OR REPLACE VARIABLE secret_key_client_id STRING DEFAULT "spn_client_id";
DECLARE OR REPLACE VARIABLE secret_key_client_secret STRING DEFAULT "spn_client_secret";
DECLARE OR REPLACE VARIABLE workspace_url STRING DEFAULT "https://e2-demo-field-eng.cloud.databricks.com";
DECLARE OR REPLACE VARIABLE connection_name STRING DEFAULT "databricks_rest_api";

In [0]:
SET VARIABLE secret_scope = :secret_scope; 
SET VARIABLE secret_key_client_id = :secret_key_client_id;
SET VARIABLE secret_key_client_secret = :secret_key_client_secret;
SET VARIABLE workspace_url = :workspace_url;
SET VARIABLE connection_name = :connection_name;

In [0]:
SELECT catalog_use, schema_use, secret_scope, secret_key_client_id, secret_key_client_secret, workspace_url, connection_name;

In [0]:
SELECT 
  CASE 
    WHEN isnull(try_secret(secret_scope, secret_key_client_id)) then "Message: Secret not found.\nPlease check that your Service Principal's details are stored in a Databricks secret with the correct secret_scope and secret_key, and that the secret_scope is accessable to the 'run_as' user of this notebook."
    ELSE "Secret for Service Principal's details are found and accessible to the 'run_as' user of this notebook.\nIf the create connection fails further in the notebook please ensure that the PAT is not expired and that the 'run_as' user has the correct permissions in Unity Catalog."
  END AS databricks_pat_secret_check
;

In [0]:
DECLARE OR REPLACE VARIABLE http_connection_stmnt STRING DEFAULT "
CREATE CONNECTION IF NOT EXISTS " || connection_name || " TYPE HTTP
 OPTIONS (
   host '" || workspace_url || "'
   ,port 443
   ,base_path '/api'
   ,client_id secret('" || secret_scope || "', '" || secret_key_client_id || "')
   ,client_secret secret('" || secret_scope || "', '" || secret_key_client_secret || "')
   ,token_endpoint '" || workspace_url || "/oidc/v1/token'
   ,oauth_scope 'all-apis'
 );
";

SELECT http_connection_stmnt;

In [0]:
EXECUTE IMMEDIATE http_connection_stmnt;